In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import preprocess

In [3]:
data = pd.read_csv('../../Data/Sentimental_Analysis/main.csv',encoding='utf-8')

In [4]:
data.head(1)

,LABEL,TWEET
0,anger,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...


In [5]:
tweet = list(data["TWEET"])
label = list(data['LABEL'])

In [6]:
file = open("../../Data/stopwords.txt","r",encoding="utf-8")
stopwords = file.read().split()
file.close()
def get_tokens(text):
    dictionary = {'د':"دكتور"}#to be continued
    words_after_split = text.split()
    for index , word in enumerate(words_after_split):
        if word in dictionary.keys():
            w = dictionary[word]
            words_after_split[index] = w
    new_words = list()
    for word in words_after_split:
        if word not in stopwords:
            new_words.append(word)
    return new_words

In [7]:
for i in range(len(tweet)):
    tweet[i] = preprocess.pre_process(str(tweet[i]))
    tokens = get_tokens(tweet[i])
    listToStr = ' '.join([str(elem) for elem in tokens])
    tweet[i] = listToStr

In [8]:
vectorizer = TfidfVectorizer (encoding='utf-8',max_df=0.6)
tweet = vectorizer.fit_transform(tweet)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(tweet, label, test_size=0.1, random_state=0)

In [10]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_jobs=2, random_state=0)
text_classifier.fit(X_train, y_train)

RandomForestClassifier(n_jobs=2, random_state=0)

In [11]:
predictions = text_classifier.predict(X_test)

In [12]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[ 981   66  421  352  169]
 [ 118  728  378  249  128]
 [ 115   57 2823  306  370]
 [ 188   84  909 1556  208]
 [ 137   67  907  238  973]]
              precision    recall  f1-score   support

       anger       0.64      0.49      0.56      1989
        fear       0.73      0.45      0.56      1601
         joy       0.52      0.77      0.62      3671
     sadness       0.58      0.53      0.55      2945
    surprise       0.53      0.42      0.47      2322

    accuracy                           0.56     12528
   macro avg       0.60      0.53      0.55     12528
weighted avg       0.58      0.56      0.56     12528

0.563617496807152


In [13]:
text_classifier.fit(tweet, label)

RandomForestClassifier(n_jobs=2, random_state=0)

In [14]:
predictions = text_classifier.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[1923   11   11   28   16]
 [  14 1547    3   21   16]
 [   8    6 3611    8   38]
 [  18   15   22 2869   21]
 [  24   11   40   21 2226]]
              precision    recall  f1-score   support

       anger       0.97      0.97      0.97      1989
        fear       0.97      0.97      0.97      1601
         joy       0.98      0.98      0.98      3671
     sadness       0.97      0.97      0.97      2945
    surprise       0.96      0.96      0.96      2322

    accuracy                           0.97     12528
   macro avg       0.97      0.97      0.97     12528
weighted avg       0.97      0.97      0.97     12528

0.9719029374201787


In [15]:
text = input()
text = preprocess.pre_process(text)
print(text)
tokens = get_tokens(text)
listToStr = ' '.join([str(elem) for elem in tokens])
text = listToStr
tweet = vectorizer.transform([text])
prediction = text_classifier.predict(tweet)
print(prediction)

سوف انتحر 
['surprise']


In [16]:
import pickle
filename = f'../../utils/sentmental_model.sav'
pickle.dump(text_classifier, open(filename, 'wb'))
filename = f'../../utils/tfidf_model.sav'
pickle.dump(vectorizer, open(filename, 'wb'))